# Parsing Strategists' Allocation Recommendation Tables

The purpose of this notebook within the [repository](...) is to demonstrate the data preprocessing algorithm used to collect, clean, and transform the stock allocation recommendation tables acquired through historic Bloomberg articles. The preprocessed data is used by the notebook [reconstructing the ssi](rough_work_analysis.ipynb) to reconstruct the Bank of America sell side indicator. More details can be found at the following [blog post](https://summernightsintohoku.wordpress.com/2024/07/28/reconstructing-the-bank-of-america-sell-side-indicator/).

In [943]:
from datetime import datetime, date
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np 
import re
import string

We begin by importing the example text file, [firms.csv](firms.csv), which contains four strategist portfolio allocation tables in the distinct formats in which they were found in Bloomberg articles. We also import a CSV file, [split_names](split_names.csv), which contains a list of the names of the various strategists and firms found within the tables, separated by the first space which appears in the name.

In [944]:
firms = pd.read_csv('firms.csv', index_col=0)['0']
split_names = pd.read_csv('split_names.csv', index_col=0)

print(firms.head())
print(split_names.head())

0    Bank of America
1               HSBC
2           JPMorgan
3        Oppenheimer
4                UBS
Name: 0, dtype: object
      First         Last
0      Bank  of Montreal
1   Goldman        Sachs
2  Deutsche         Bank
3    Stifel     Nicolaus
4    Morgan      Stanley


Here is an example of the table contained within the first article in the text file.

In [945]:
forecasts = open(f'example_tables.txt', 'r').read()
print(forecasts[:forecasts.index('=========\n\n')])

NEWS

Strategists’ S&P 500 Index Estimates for Year-End 2024 (Table)
Fri Jan 19 14:57:16 2024 UTC

 By Lu Wang  (Bloomberg) -- The table presents estimates from strategists for
where the S&P 500 Index will finish 2024 and how much profit companies will
generate.                           S&P 500 Index  *T
|    2024 Close     |    2024 EPS
================================================================ Average
|              4,867|            $234 Median                     |
4,950|            $235 High                       |              5,200|
$250 Low                        |              4,200|            $221 No. of
replies             | 20                |              20 *T
Individual Estimates  *T
================================================================  Firm         |
Strategist     |2024 Close | 2024 EPS
================================================================ Bank of America
|Savita Subramanian  |      5,000|     $235 Barclays             |Venu Krishna
|    

Our first step is to fix any spelling mistakes and set any naming conventions which will be uniformly applied throughout the articles.

In [946]:
patterns = {r'Citi(?!g)' : 'Citigroup', r'JPMorgan Chase|J.P.(?:\s|\n)Morgan|JP(?:\s|\n)Morgan' : 'JPMorgan', 'Banc' : 'Bank', r'By(?:\n|\s)Committee' : 'Committee', r'Bank(\s|\n)of(\s|\n)Montreal' : 'BMO', 'V22' : '22V', 'Evecore' : 'Evercore', 'Robert W. Baird' : 'RW Baird', r'Global|Advisors|Research|LLC|Inc|Capital|Markets|&(\s|\n)Co\.|Warburg' : '', 'NEWS\n\n' : '', r'S&P\s{2,}500' : 'S&P 500'}

def fix_mistakes(forecasts):
    corrections = []
    for mistake in patterns:
        count = 0
        fix = patterns[mistake]
        count = len(re.findall(mistake, forecasts)) 
        if count>0:
            corrections.append([re.sub(r'\n', '', mistake),count,fix])
        forecasts = re.sub(mistake, fix, forecasts)
    print(f'Corrections made:')
    for correction in corrections:
        print(f'\'{correction[0]}\' changed to \'{correction[2]}\' ({correction[1]}x)')
    return forecasts

In [947]:
forecasts = fix_mistakes(forecasts)

Corrections made:
'Citi(?!g)' changed to 'Citigroup' (1x)
'JPMorgan Chase|J.P.(?:\s|\n)Morgan|JP(?:\s|\n)Morgan' changed to 'JPMorgan' (1x)
'Banc' changed to 'Bank' (1x)
'By(?:\n|\s)Committee' changed to 'Committee' (1x)
'Bank(\s|\n)of(\s|\n)Montreal' changed to 'BMO' (3x)
'Global|Advisors|Research|LLC|Inc|Capital|Markets|&(\s|\n)Co\.|Warburg' changed to '' (8x)
'NEWS' changed to '' (1x)
'S&P\s{2,}500' changed to 'S&P 500' (1x)


In [948]:
articles = re.split('=========\n\n', forecasts)
print(f'There are {len(articles)} articles.')

There are 4 articles.


We write a function to extract author of each article and the date it was published:

In [949]:
def get_info(articles):
   pattern_authors = r'(?<=By\s).*?(?:\s|\n).*?(?=(?:\s\s|\n))'
   authors = []
   for table in articles:
      author = re.findall(pattern_authors, table, re.DOTALL)
      if not author:
         author = ['None']
      authors.append(author[0])
   dates = [datetime.strptime(re.split(r'\n', article)[1], '%a %b %d %H:%M:%S %Y %Z').date() for article in articles] 
   df = pd.DataFrame({'Date' : dates, 'Author' : authors})
   df['Date'] = pd.to_datetime(df['Date'])
   return df

info = get_info(articles)
info

,Date,Author
0,2024-01-19,Lu Wang
1,2016-06-01,Oliver Renick
2,2009-11-23,Mary Childs
3,2003-01-03,None


The following function separates the articles, extracts the table(s) from each article, and the footnotes from each table, storing them into separate lists: 
1. The first table in each article is stored in the variable `table_texts`.
2. The second table in each article, if it exists, is stored in `a_table_texts`.
3. The table footnotes (for both articles) are stored in `footnote_texts`.

In [950]:
def separate_article(articles):
    table_texts, a_table_texts, footnote_texts = [], [], []
    pattern = r'Firm.*?(?=Mean|Averages|\*T|AVERAGE|--)|\s[a-z][\.\s][\s|\-].*?(?=\*T|Firm|To contact|For Related|\-\-[A-Z])'
    for j, article in enumerate(articles):
        a_table_texts.append('')
        footnote_texts.append('')
        article_segments = re.findall(pattern, article, re.DOTALL)
        footnote_text = ''
        for i, text in enumerate(article_segments):
            if i == 0:
                table_texts.append(article_segments[0])
            elif re.findall(r'Firm\s\s', text):
                a_table_texts[j] = text
            elif re.fullmatch(r'[a-z]\.\s|[a-z]\s-',text.strip()[:3]):
                footnote_text += re.sub(r'\n(?!=\s)|(?<!=\s)\n', ' ', text.strip())
        if footnote_text:
            footnote_texts[j] += footnote_text
    return table_texts, a_table_texts, footnote_texts

table_texts, a_table_texts, footnote_texts = separate_article(articles)

We partition the string of footnotes from each article into separate lists of individual footnotes.

In [951]:
# Convert each string of footnotes in footnote_texts to a list of footnotes
pattern = r'\s{1,}(?=[a-z][\.\s][\s\-])|.(?=Note:)'

footnote_lists = []
for footnote_text in footnote_texts:
    footnote_lists.append(re.split(pattern, footnote_text))

footnote_lists[3]

["a - The firm doesn't have a forecast.",
 'b - 12-month forecast.',
 'c - 2003 year-end forecast.',
 'd - CIBC recommends placing 3 percent in real estate investments, including real estate investment trusts and shares of development companies.',
 "e - The firm doesn't have an asset allocation model.",
 'f - Deutsche Bank forecasts a range of 9000 to 9500 for the Dow Average by the end of 2003.',
 'g - Goldman Sachs recommends placing 3 percent in commodities.',
 'h - Jeffrey Applegate has left the firm. No replacement has been named yet.',
 "i - This is UBS 's ``current fair market value'' target for the S&P 500. The firm doesn't give year-end or 12-month forecasts."]

The following function picks out keywords in table footnotes which indicate the footnotes are irrelevant for our needs of the data. Values associated with those footnotes will be included within our final dataframes. Footnotes which are not deleted indicate values which must be discarded.

In [952]:
#Remove footnotes which are unimportant to our analysis from footnote_lists and identify labels of nondiscarded footnotes.
def contains_word(words,text):
    for word in words:
        if re.findall(word, text):
            return True
    return False

boring_words = ['recommends', 'produced by', 'set by', 'sets the Dow', 'sets the S&P', 'from research reports', 'were made by', r'for.*?(?=long).*?(?=term).*?investors', 'Joint strategists', 'bond allocation breakdown', 'resigned', 'left the firm', 'Changes from', 'changed his', 'replacement', 'doesn\'t have a', 'Editor', 'Related News']

def fix_footnote_lists(footnote_list):
    footnote_letters, footnotes = [], [[] for j in range(2)]
    print('Footnotes removed:')
    for j, footnote_list in enumerate(footnote_lists):
        print(f'\nTable {j}:')
        footnote_letters.append([])
        for footnote in footnote_list:
            if footnote:
                if contains_word(boring_words, footnote):
                    print(footnote)
                else:
                    footnote_letters[j].append(footnote[0])
                    if footnote in footnotes[0]:
                        index = footnotes[0].index(footnote)
                        footnotes[1][index].append(j)
                    else:
                        footnotes[0].append(footnote)
                        footnotes[1].append([j])
    return footnote_letters, footnotes

footnote_letters, footnotes = fix_footnote_lists(footnote_lists)

print('\nFootnotes kept:')
for i, footnote in enumerate(footnotes[0]):
    print(f'{i}.',footnote)

Footnotes removed:

Table 0:

Table 1:

Table 2:
c. Citigroup’s estimate for S&P 500 profit is produced by Steven Wieting. 
Note: Oppenheimer’s Brian Belski forecasts the S&P 500 will be at 1,140 in 12 months and estimates earnings for companies in the index during that time will be $60.50 a share.d. The asset allocation projections at UBS come from Larry Hatheway, who also recommends investing 5.5 percent in commodities, 5 percent in real estate and 2.5 percent in inflation-indexed securities.

Table 3:
a - The firm doesn't have a forecast.
d - CIBC recommends placing 3 percent in real estate investments, including real estate investment trusts and shares of development companies.
e - The firm doesn't have an asset allocation model.
g - Goldman Sachs recommends placing 3 percent in commodities.
h - Jeffrey Applegate has left the firm. No replacement has been named yet.

Footnotes kept:
0. a. Oppenheimer and Citigroup also recommend investing 5 percent and 16 percent in alternative inv

In [953]:
# Identify problemeatic footnotes (i.e., those whose associated vales we do not want in our dataset') by their index in 'footnotes'.
bad_footnotes = [3,5] 

In [954]:
def bad_footnote_letters(bad_footnotes):
    bad_footnote_letters = [[] for j in range(len(articles))]
    for i in bad_footnotes:
        footnote = footnotes[0][i]
        indices = footnotes[1][i]
        for j in indices:
            bad_footnote_letters[j].append(footnote[0])
    return bad_footnote_letters

bad_footnote_letters = bad_footnote_letters(bad_footnotes)

In [955]:
def wrap_spaces(match):
    return '  ' + match.group(0) + '  '

def add_hyphen(match):
    return '-' + match.group(0) 

def l_space(match):
    return '  ' + match.group(0) 

def fix_body(text: str) -> str:
    text = re.sub(r'(?<=[a-z])[0-9]', l_space, text)
    text = re.sub(r'[\s\-][a-z](\s|$)', wrap_spaces, text) 
    text = re.sub(r'(?<=\s|\n)[a-z](?=\s|\n)', add_hyphen, text)
    text = re.sub(r'[\n\|]', '  ', text) 
    text = re.sub(r'[a-z]\s{0,1}(?=[$0-9])|(?<=[%0-9])\s{0,1}[A-Z](?!\s)', l_space, text)
    return text

def fix_header(text: str) -> str:
    text = re.sub(r'\s(?!Close|EPS|500)', wrap_spaces, text)
    text = re.sub(r'\n|\|', ' ', text)
    return text

The following helper functions convert the table(s) found in each article located in `table_texts` and `a_table_texts` into dataframes via the following process (in what follows, let `k` denote the index of the article in the file [example_tables.txt](example_tables.txt) which the tables are contained within):
1. The function `split_table_text` separates the header of each table from the `body'. 
2. The function `get_table_type` determines which (of the 4 possible) table types the inputted article falls into:
    1. Article contains one table, with header

        "Firm | Strategist | Year-end S&P500 Close | Year-end EPS".
        
    2. Article contains one table, with header 

        "Firm | Strategist | Stock | Bonds | Cash"
        
        with the latter three headings referring to allocation recommendations.
    3. Article contains two tables, the first in the form of type 1 above, and the second in the form of type 2 above (without the 'Strategist' heading).
    4. Article contains one table, with header

        "Firm | Strategist | Stock | Bonds | Cash | Year-end Dow Close | Year end S&P500 Close".
3. The function `remove_footnotes` deletes all footnotes appearing in the table which are not contained in the `k`th index of `bad_footnote_letters'.
4. The function `fix_names' 

In [956]:
#helper functions

def split_table_text(table_text, year):
    if '=' in table_text:
        pattern = r'={1,}'
    elif 'S&P 500' in table_text: 
        pattern = r'(?<=S&P 500)\s'
    else:
        pattern = r'(?<=Cash)\s'
    split = re.split(pattern, table_text)
    header, body = fix_header(split[0].strip()), fix_body(split[1].strip())
    columns = re.split(r'\s{2,}', header)
    if 'S&P 500' in columns:
        index = columns.index('S&P 500')
        columns[index] = f'Year-end Close' 
    if f'{year} Close' in columns:
        index = columns.index(f'{year} Close')
        columns[index] = f'Year-end Close' 
    if f'{year} EPS' in columns:
        index = columns.index(f'{year} EPS')
        columns[index] = 'Year-end EPS' 
    return columns, body

def get_table_type(a_table_text, columns):
    if a_table_text:
        return 3
    elif 'Year-end Close' in columns:
        if 'Stocks' in columns:
            return 4
        else:
            return 1
    else:
        return 2

def is_correct_type(word, header):
    if word:
        if 'Close' in header:
            return bool(re.fullmatch(r'[0-9]{3,4}|[0-9]{1,2},[0-9]{3}', word))
        if 'Dow' in header:
            return bool(re.fullmatch(r'[0-9]{2},{0,1}[0-9]{3}|[8-9],{0,1}[0-9]{3}', word))
        elif header in ('Stocks', 'Bonds', 'Cash', 'Commodities'):
            return bool(re.fullmatch(r'[0-9]{1,3}\.[0-9]|[0-9]{1,2}|100', word))
        elif 'EPS' in header:
            return '$' == word[0]
        elif header == 'Strategist':
            return len(re.findall(r'[A-Z]', word)) > 0
        else:
            return 'Error'
    return True



def remove_footnotes(line, k):
    matches = re.findall(r'(?<=\-)[a-z]', line)
    for letter in matches: 
        if letter not in bad_footnote_letters[k]:
            line = re.sub(f'-{letter}', '', line) 
    line = re.sub(r'-[a-z]', wrap_spaces, line).strip()   
    return line

def fix_names(values):
    for i, row in split_names.iterrows(): 
        first = row['First']
        last = row['Last']
        spacing = " "
        if first in values and last in values and values.index(last) == values.index(first)+1:
            if first[-1] == '-':
                spacing = ""
            values = values[:values.index(first)] + [first + spacing + last] + values[values.index(last)+1:]    
    return values

def remove_footnotes2(values, k):
    for i in bad_footnotes: 
        footnote, indices = footnotes[0][i], footnotes[1][i]
        if k in indices:
            letter = footnote[0]
            while f'-{letter}' in values:
                index = values.index(f'-{letter}')
                values = values[:index-1] + values[index+1:]
    return values

def get_values(table_text, k):  
    lines = [line for line in re.split(r'\n|=.*?(?=\s)|-{2,}.*|\|', table_text.strip()) if line.strip()]
    values = []
    for i, line in enumerate(lines):
        line = remove_footnotes(line,k)
        values.extend(re.split(r'\s{2,}', line))
    values = [value for value in values if value]
    values = remove_footnotes2(fix_names(values), k)
    return values


def form_rows(values):
    indices = []
    for i, value in enumerate(values):
        if value in firms.values:
            indices.append(values.index(value)) 
    rows = [values[indices[i]:indices[i+1]] for i in range(len(indices)-1)]  
    rows.append(values[indices[-1]:])
    return rows

def fix_table(table, columns):
    row_size = len(columns)
    new_table = []
    for j, row in enumerate(table):
        if len(row) < row_size:
            for i, value in enumerate(row):
                if not is_correct_type(value, columns[i]):
                    if i < len(row) - 1:
                        row.insert(i,None)
                    elif i < len(columns)-1:
                        count = [is_correct_type(value, columns[j]) for j in range(i+1,len(columns))].index(True)
                        for _ in range(count+1):
                            row.insert(i, None)
        new_table.append(row + [None for i in range(row_size-len(row))])
    return np.array(new_table)

def fill_table(table: np.ndarray, firms: np.ndarray) -> np.ndarray:
    test = list(table[:,0])
    for i, firm in enumerate(firms):
        if not firm in test:
            row = np.array([firm] + [np.nan] * 3)
            table = np.insert(table, i, row, axis=0)
    return table

def text_to_table(values: list, columns: list) -> np.ndarray:
    return fix_table(form_rows(values), columns)

def format_df(df: pd.DataFrame) -> pd.DataFrame:
    df.fillna(np.nan, inplace=True)
    df['Stocks'] = df['Stocks'].str.replace('%','').astype('float')
    df['Bonds'] = df['Bonds'].str.replace('%','').astype('float')
    df['Cash'] = df['Cash'].str.replace('%','').astype('float')
    return df

In [957]:
def text_to_dataframe(table_text, a_table_text, year, k):
    columns, body = split_table_text(table_text, year)
    table_type = get_table_type(a_table_text, columns)
    table_values = get_values(body, k)
    table = text_to_table(table_values, columns)
    if table_type == 3: 
        a_columns, a_body = split_table_text(a_table_text, year)
        a_table_values = get_values(a_body, k)
        a_table = text_to_table(a_table_values, a_columns)
        columns += a_columns[1:]
        firms = table[:,0]
        a_table = fill_table(a_table, firms)
        a_table = a_table[:,1:]
        table = np.concatenate([table, a_table], axis=1)
    df = pd.DataFrame(table, columns = columns)
    columns = ['Date', 'Author'] + columns 
    df[['Date','Author']] = info.loc[k].values 
    df = df[columns]
    return(df)

In [958]:
df_list = []
for k, table_text in enumerate(table_texts):
    a_table_text = a_table_texts[k]
    year = info['Date'][k].year
    letters = footnote_letters[k]
    df_list.append(text_to_dataframe(table_text, a_table_text, year, k))

df = pd.concat(df_list, axis=0, ignore_index=True)
df = format_df(df)
df

,Date,Author,Firm,Strategist,Year-end Close,Year-end EPS,Stocks,Bonds,Cash,Dow
0,2024-01-19 00:00:00,Lu Wang,Bank of America,Savita Subramanian,"5,000",$235,NaN,NaN,NaN,NaN
1,2024-01-19 00:00:00,Lu Wang,Barclays,Venu Krishna,"4,800",$233,NaN,NaN,NaN,NaN
2,2024-01-19 00:00:00,Lu Wang,BMO,Brian Belski,"5,100",$250,NaN,NaN,NaN,NaN
3,2024-01-19 00:00:00,Lu Wang,Cantor Fitzgerald,Eric Johnston,"4,400",$225,NaN,NaN,NaN,NaN
4,2024-01-19 00:00:00,Lu Wang,Citigroup,Scott Chronert,"5,100",$245,NaN,NaN,NaN,NaN
5,2024-01-19 00:00:00,Lu Wang,Deutsche Bank,Binky Chadha,"5,100",$250,NaN,NaN,NaN,NaN
6,2024-01-19 00:00:00,Lu Wang,Evercore ISI,Julian Emanuel,"4,750",$221,NaN,NaN,NaN,NaN
7,2024-01-19 00:00:00,Lu Wang,Fundstrat,Tom Lee,"5,200",$240,NaN,NaN,NaN,NaN
8,2024-01-19 00:00:00,Lu Wang,Goldman Sachs,David Kostin,"5,100",$237,NaN,NaN,NaN,NaN
9,2024-01-19 00:00:00,Lu Wang,JPMorgan,Dubravko Lakos-Bujas,"4,200",$225,NaN,NaN,NaN,NaN
